In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = './Data/'
start_date = '2018-04-12'

In [3]:
company_name = pd.read_csv(path + 'Company Group.csv', header = [0])
company_code = company_name['Company Code on Yahoo Finance'].values.tolist()
directory = path + 'ASX_stockconsidering/'
Group_dict = []
for i in company_name['Group'].values.tolist():
    if i not in Group_dict:
        Group_dict.append(i)

In [4]:
def read_stock(file_name, directory = directory):
    stock = pd.read_csv(directory + file_name, header = [0])
    index = pd.to_datetime(stock['Date'])
    stock.drop('Date', axis = 1)
    stock.index = index
    # To predict the Adjusted Close Price of the Stock
    stock = stock[['Adj Close']]
    return stock

def read_stockfiles(path = directory):
    All_df = []
    files = [f for f in os.listdir(path) if f.endswith('.csv')]
    delete_list = []
    for i in company_code:
        if (i + '.csv' not in files):
            delete_list.append(i)
    for i in delete_list:
        company_code.remove(i)
    
    for name in files:
        stock = read_stock(name)
        if (stock.isna().any().any()):
            print("NA show")
            stock = stock.fillna(method = 'ffill')
        All_df.append(stock[start_date:])
    return All_df, files

All_stocks, files = read_stockfiles()

NA show
NA show


In [5]:
# Create the directories
for directory_name in Group_dict:
    os.makedirs(path + 'Grouped/' + directory_name, exist_ok=True)

In [6]:
for stock in range(len(All_stocks)):
    stock_name = files[stock]
    stock_name = stock_name.replace('.csv', "")
    pos = company_name.loc[company_name['Company Code on Yahoo Finance'] == stock_name].index[0]
    group = company_name.loc[pos, 'Group']
    the_stock = All_stocks[stock]
    the_stock.to_csv(path + 'Grouped/' + group + '/' + stock_name + '.csv')